# 🧪 MS/MS Spectra Modification Classifier with Transformers

This notebook builds and trains a deep learning model designed to detect and classify *post-translational modifications (PTMs)* in MS/MS spectra from shotgun proteomics. The input is MS/MS spectra in `mgf` format and the classifier is based on a hybrid CNN-Transformer architecture.

---

### 🧠 Objectives

- **Multi-class Classification**: If modified, predict the specific type:
  - Unmodified
  - Oxidation
  - Phosphorylation

---

### 🔧 Environment Setup

The following libraries and paths are configured:

#### 📦 Core Libraries
- `torch`, `torch.nn`, `torch.optim`: PyTorch for neural network construction and training.
- `numpy`, `random`, `os`, `sys`: Utilities for array operations, randomness, and file handling.
- `math`, `datetime`, `logging`: Math functions, timestamping, and logging system.
- `matplotlib.pyplot`: (optional) Visualization.
- `scikit-learn`: Evaluation metrics and dataset splitting.


#### 🛠️ Path Configuration
- Adds the dataset directory on Google Drive to the system path to ensure data files can be accessed during training and evaluation.

---

### 🧬 Pipeline Overview

This project includes the following components:
- **MGF File Parsing**: Custom loader to extract raw spectra from `.mgf` files.
- **Spectral Preprocessing**: Converts spectra into binned, normalized vector representations.
- **Metadata Normalization**: Processes and scales parent ion mass (`pepmass`) for model input.
- **Transformer-Based Model**: Hybrid neural architecture combining CNNs, self-attention, and metadata fusion.
- **Training & Evaluation**: Loop with weighted loss, custom metrics, logging, and model checkpointing.

This setup is tailored for high-performance PTM classification while maintaining compatibility with Google Colab workflows and GPU acceleration tuned using Optuna.


In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 35.9 MB/s eta 0:00:00


In [ ]:
#Set up the enviorment imports and paths that are necessary for the processing of the cells

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('content/drive/MyDrive/data/balanced_dataset')  # Add the folder containing main.py to sys.path
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import os
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, precision_recall_curve
import matplotlib.pyplot as plt
import logging
from datetime import datetime
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import classification_report
import torch.nn.functional as F
from torch.nn import SiLU
import optuna
import joblib
import optuna
import torch
import numpy as np
import random
import os
import math
from datetime import datetime
from sklearn.model_selection import train_test_split


Mounted at /content/drive


## 📂 DatasetHandler class for loading MGF files

This section defines the `DatasetHandler` class responsible for managing the loading and iteration over `.mgf` files containing MS/MS spectra.
Loading only one `.mgf` at a time in order to make the pipeline scalable without running avoiding memory memory issues.

---

### 📦 `DatasetHandler` Overview

The `DatasetHandler` class provides a memory-efficient way to iterate through `.mgf` files stored in a directory. It supports:

- **Shuffling input files** to randomize data order across training loops.
- **Per-file usage tracking** with `MAX_FILE_PASSES`, ensuring that no file is overused during training.
- **Controlled looping** over the dataset using `num_loops` to allow multiple training epochs without data reloading.

---

### 🧩 Key Components make this under the code explaining how to use it, make it like an example under evrything

#### 🔧 Initialization
```python
handler = DatasetHandler(input_dir="/path/to/mgf", num_loops=1)

In [ ]:
#Setting up the dataset handler class that handles the input

MAX_FILE_PASSES = 1 # Max times a file can be used before being ignored

class DatasetHandler:
    def __init__(self, input_dir, num_loops=1):
        """
        Initialize the dataset handler.

        Args:
            input_dir (str): Path to the directory containing split MGF files.
            num_loops (int): Number of times the dataset should be iterated.
        """
        self.files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.mgf')]
        self.files = random.sample(self.files, len(self.files))  # Shuffle files
        self.file_usage_counter = {f: 0 for f in self.files}
        self.num_loops = num_loops
        self.loop_count = 0

    def get_next_file(self) -> list:
        """
        Load one MGF file at a time into RAM and return all spectra from it.

        Returns:
            list of dict: Each dict contains a full spectrum from the selected file and metadata information.
        """
        while self.loop_count < self.num_loops:
            available_files = [f for f in self.files if self.file_usage_counter[f] < MAX_FILE_PASSES]

            if not available_files:
                self.loop_count += 1
                if self.loop_count < self.num_loops:
                    print("Restarting dataset loop...")
                    self.file_usage_counter = {f: 0 for f in self.files}
                    continue
                else:
                    print("All dataset loops completed.")
                    return None  # No more data

            file = random.choice(available_files)
            print(f"Processing file: {file}")

            spectra = []
            spectrum_data = {}  # Define it once before parsing

            with open(file, 'r') as f:
                for line in f:
                    line = line.strip()

                    if line.startswith("TITLE="):
                        spectrum_data["title"] = line.split("=", 1)[1]
                    elif line.startswith("PEPMASS="):
                        spectrum_data["pepmass"] = float(line.split("=", 1)[1].split()[0])
                    elif line.startswith("CHARGE="):
                        spectrum_data["charge"] = int(line.split("=", 1)[1].replace("+", "").replace("-", ""))
                    elif line == "BEGIN IONS":
                        spectrum_data = {"mz_values": [], "intensity_values": [], "title": ""}
                    elif line == "END IONS":
                        title = spectrum_data.get("title", "").strip()
                        mz_vals = spectrum_data.get("mz_values", [])
                        int_vals = spectrum_data.get("intensity_values", [])

                        if not title:
                            print(f"[SKIPPED] Spectrum missing TITLE in file: {file}")
                        elif not mz_vals:
                            print(f"[SKIPPED] Spectrum '{title}' has no m/z values in file: {file}")
                        elif not int_vals:
                            print(f"[SKIPPED] Spectrum '{title}' has no intensity values in file: {file}")
                        else:
                            spectra.append(spectrum_data)

                    else:
                        try:
                            mz, intensity = map(float, line.split()[:2])
                            spectrum_data["mz_values"].append(mz)
                            spectrum_data["intensity_values"].append(intensity)
                        except ValueError:
                            pass  # Ignore unparseable lines

            self.file_usage_counter[file] += 1  # Update file usage count

            if spectra:
                return spectra, file  # Return all spectra from the file

        print("All spectra processed.")
        return None



# ⚙️ Dense Vector Binning for 1D CNN Input  
This section defines the updated preprocessing pipeline for converting annotated MS/MS spectra into dense, fixed-length vectors. These are tailored for use in models such as CNNs or hybrid CNN-Transformer architectures.

## 🔧 Functions:

### `bin_spectra_to_dense_vectors`  
Converts a list of spectra into fixed-length vectors by:  
- **Binning the m/z values** across a specified range (`mz_min` to `mz_max`) into `num_bins`.  
- Each bin holds the **intensity sum of peaks** falling into that m/z range.  
- Applies **sliding window normalization**:  
  The m/z axis is divided into fixed-size windows (e.g., 200 m/z), and intensities within each window are normalized individually to the [0, 1] range. This preserves local signal structure and prevents domination by high-intensity regions.

### `process_spectra_with_handler`  
Processes a batch of spectra by:  
- Logging and skipping spectra with empty or invalid m/z or intensity values.  
- Using the above function to apply binning and **sliding window normalization**.  
- Skipping spectra with no signal after binning (i.e., zero-vector).  

Returns a list of valid, normalized dense vectors for CNN input and logs the total number of skipped spectra.

## 📦 Output Format:  
Each spectrum becomes a 1D `np.array` of shape `(num_bins,)` with `float32` values.  

The final output is either:  
- a stacked `np.ndarray` of shape `(batch_size, num_bins)` when using `bin_spectra_to_dense_vectors` directly on a list, or  
- a list of valid vectors (1 per spectrum) when using `process_spectra_with_handler`.
 used.

In [ ]:
def bin_spectra_to_dense_vectors(spectra_data, num_bins=5000, mz_min=100.0, mz_max=2200.0, window_size=200.0):
    """
    Converts spectra into dense, fixed-length binned vectors suitable for 1D CNN input with sliding window normalization.

    Parameters:
    - spectra_data: List of spectra dicts with 'mz_values' and 'intensity_values'.
    - num_bins: Number of bins to divide the m/z range [mz_min, mz_max] into.
    - mz_min: Minimum m/z value for binning.
    - mz_max: Maximum m/z value for binning.
    - window_size: Size of m/z window for normalization (default is 200.0).

    Returns:
    - np.ndarray of shape (batch_size, num_bins) with per-spectrum normalized intensities.
    """
    bin_edges = np.linspace(mz_min, mz_max, num_bins + 1)
    binned_spectra = []

    for spectrum in spectra_data:
        mz_values = np.array(spectrum['mz_values'])
        intensity_values = np.array(spectrum['intensity_values'])

        if len(mz_values) == 0 or len(intensity_values) == 0:
            binned_spectra.append(np.zeros(num_bins, dtype=np.float32))
            continue

        # Create an array to hold the binned intensities (fixed size)
        binned_intensity = np.zeros(num_bins)

        # Iterate over windows of m/z values
        for window_start in np.arange(mz_min, mz_max, window_size):
            window_end = window_start + window_size
            window_mask = (mz_values >= window_start) & (mz_values < window_end)
            window_mz_values = mz_values[window_mask]
            window_intensity_values = intensity_values[window_mask]

            if len(window_mz_values) > 0:
                # Bin the intensities for this window
                binned_window_intensity, _ = np.histogram(window_mz_values, bins=bin_edges, weights=window_intensity_values)

                # Normalize the binned intensities within this window
                min_val = binned_window_intensity.min()
                max_val = binned_window_intensity.max()
                range_val = max_val - min_val if max_val != min_val else 1e-6
                normalized_binned_window = (binned_window_intensity - min_val) / range_val

                # Add the normalized intensities to the final vector (same size as before)
                binned_intensity += normalized_binned_window

        binned_spectra.append(binned_intensity.astype(np.float32))

    return np.stack(binned_spectra)  # Shape: (batch_size, num_bins)


def process_spectra_with_handler(spectra_batch, num_bins=1000, window_size=200.0):
    """
    Processes spectra batch and returns a list of 1D CNN-ready vectors (one per spectrum),
    with sliding window normalization applied.
    """
    spectrum_vectors = []
    skipped_spectra = 0

    for idx, spectrum in enumerate(spectra_batch):
        title = spectrum.get("title", f"unnamed_{idx}")
        mz_values = np.array(spectrum['mz_values'])
        intensity_values = np.array(spectrum['intensity_values'])

        if mz_values.size == 0 or intensity_values.size == 0:
            print(f"[SKIPPED] Empty m/z or intensity array: '{title}'")
            skipped_spectra += 1
            continue

        # Call the binning function with windowed normalization
        binned_spectrum = bin_spectra_to_dense_vectors([spectrum], num_bins=num_bins, window_size=window_size)

        # Ensure only valid (non-zero) spectra are added
        if np.sum(binned_spectrum) == 0:
            print(f"[SKIPPED] Zero intensity after binning: '{title}'")
            skipped_spectra += 1
            continue

        spectrum_vectors.append(binned_spectrum[0])  # Extract the vector

    print(f"Total skipped spectra: {skipped_spectra}")
    return spectrum_vectors


## 🔬 Normalize Parent Ion Mass (PEPMASS)

This module provides utilities to **extract sequences**, **convert observed m/z to monoisotopic neutral mass** (if needed), and **normalize parent ion values** into the range [0, 1].

---

### 🎯 Objectives (current implementation)

- **Extract** peptide sequence from the beginning of the `TITLE` field.  
- **Convert** PEPMASS from **observed m/z** to **monoisotopic single charged mass** when `assume_observed=True`.  
- **Normalize** the parent ion mass into \[0, 1\] using global bounds from `min_max_dict`.



### 🧩 Key Functions

#### 🔹 `extract_sequence_from_title(title: str) -> str`
Extracts the peptide sequence from the `TITLE`.  
Assumes the sequence is the **first token** (before the first space).

**Example**
```python
TITLE = "GWSMSEQSEESVGGR 2,S,Phospho"
extract_sequence_from_title(TITLE)
# → "GWSMSEQSEESVGGR"
```

🔹 `observed_to_monoisotopic(observed_mz: float, charge: int) -> float`

Converts observed precursor **m/z** into **monoisotopic neutral mass**:

$$
\text{mono\_mass} = z \cdot \text{m/z} - (z - 1)\cdot \text{PROTON\_MASS}
$$

Uses `PROTON_MASS = 1.007276`.

---

#### 🔹 `normalize_parent_ions(data, min_max_dict, assume_observed=True) -> list[float]`

Normalizes parent ion values to the range \$0, 1\$.

* **Inputs per spectrum (dict):**

  * `"pepmass"`: precursor value
  * `"charge"`: integer charge state

* **Behavior:**

  1. If `assume_observed=True`:

     * Converts `"pepmass"` (observed m/z) → monoisotopic neutral mass.
  2. If `assume_observed=False`:

     * Uses `"pepmass"` directly (assumed monoisotopic).
  3. Normalizes with:

     $$
     \text{norm} = \frac{parent\_ion - min}{max - min}
     $$
  4. Clamps results into \$0, 1\$.
  5. Missing metadata → returns `0.0`.

**Example**

```python
min_max = {"min": 500.00, "max": 6000.00}
normalized = normalize_parent_ions(spectra, min_max, assume_observed=True)
```

---

### ✅ Output

Returns:

```python
[List of float values between 0 and 1]
```

---

### ⚠️ Notes

* Requires `"min"` and `"max"` keys in `min_max_dict`.
* Missing or invalid metadata defaults to **0.0**.
* No theoretical mass calculation or spectrum validation is performed here.




In [ ]:
PROTON_MASS = 1.0072764665789
H2O_MASS = 18.01056

def extract_sequence_from_title(title: str) -> str:
    """
    Extracts the peptide sequence from the TITLE string.
    Assumes the sequence is the first word, before the first space.
    """
    if not isinstance(title, str) or not title.strip():
        return ""
    return title.strip().split(" ")[0]  # safe even with extra spaces



def observed_to_monoisotopic(observed_mz, charge):
    return charge * observed_mz - (charge - 1) * PROTON_MASS



def normalize_parent_ions(data, min_max_dict, assume_observed=True):
    """
    Normalize parent ions to the range [0, 1].

    If assume_observed=True, converts PEPMASS (observed m/z) to monoisotopic mass before computing normalization.
    """
    normalized = []

    for spectrum in data:
        pepmass = spectrum.get("pepmass", None)
        charge = spectrum.get("charge", None)

        if pepmass is None or charge is None:
            normalized.append(0.0)
            continue

        if assume_observed:
            mono_mass = observed_to_monoisotopic(pepmass, charge)
            parent_ion = mono_mass
        else:
            parent_ion = pepmass  # Already monoisotopic

        # Normalize to [0, 1]
        pepmass_min = min_max_dict["min"]
        pepmass_max = min_max_dict["max"]
        norm = (parent_ion - pepmass_min) / (pepmass_max - pepmass_min)
        normalized.append(max(0, min(1, norm)))

    return normalized




### 🧬 Combine Spectra with Parent Ion Mass, change the model to always recieve monoistopic mass inetas of obserd mass like we currently do.


This function constructs the final **input representation** for the neural network by pairing each processed spectrum with its corresponding normalized parent ion mass.

---

### ⚙️ `combine_features(...)`

#### **Purpose**
Aggregates spectral and precursor metadata into a unified format, ready to be passed into the model during training or evaluation.

---

### 🔄 Process Flow

1. **Spectral Preprocessing**
   - Calls `process_spectra_with_handler(...)` to:
     - Apply binning and normalization.
     - Generate a dense, fixed-length vector for each spectrum.
   - Result: `spectra_vectors` — a list of shape `[batch_size, num_bins]`.

2. **Parent Ion Normalization**
   - Invokes `normalize_parent_ions(...)` to:
     - Convert precursor monoisotopic mass to observed mass.
     - Normalize to a range of [0, 1] using dataset-specific bounds.
   - Result: `parent_ions` — a list of length `[batch_size]`.

3. **Validation**
   - Verifies alignment between spectrum vectors and parent ion list.
   - Logs an error and aborts if lengths mismatch.

4. **Zipping**
   - Combines each spectrum vector and its corresponding normalized parent ion into a tuple:
     ```python
     (spectrum_vector, normalized_parent_ion)
     ```

---

### 📤 Output Format

```python
[
  (spectrum_vec₁, pepmass₁),
  (spectrum_vec₂, pepmass₂),
  ...
]

In [ ]:
def combine_features(data, pepmass_min_max, num_bins, window_normaliation_size, assume_observed):
    """
    Converts spectra + metadata into model input tuples:
        (binned spectrum, normalized parent ion mass)
    """

    spectra_vectors = process_spectra_with_handler(data, num_bins, window_normaliation_size)
    if not spectra_vectors:
        return None

    parent_ions = parent_ions = normalize_parent_ions(
    data, pepmass_min_max, assume_observed=assume_observed)


    if len(spectra_vectors) != len(parent_ions):
        print("❌ Mismatch between spectra and parent ions.")
        return None

    return list(zip(spectra_vectors, parent_ions))


### 🏷️ Label Spectra Based on Modifications

This function performs **automatic labeling** of MS/MS spectra for supervised learning, based on the content of the `TITLE` field in each spectrum's metadata.

---

### 🧠 Purpose

Assigns integer labels to each spectrum in a batch according to the presence of post-translational modification (PTM) keywords in the title:

- `0` → **Unmodified**
- `1` → **Oxidation** (if the word `"oxidation"` appears in the title)
- `2` → **Phosphorylation** (if the word `"phospho"` appears in the title)

The result is a list of labels aligned with the order of input spectra — suitable for classification tasks using `CrossEntropyLoss`, `BCEWithLogitsLoss`, or one-hot encoding strategies.

---

### ⚙️ Logic

For each spectrum in the input list:
1. Checks that the entry is a dictionary.
2. Extracts the `title` and converts it to lowercase.
3. Searches for PTM-related keywords.
4. Defaults to `0` if no match or invalid format.

---

### 📤 Output Format

Returns:
```python
[0, 2, 1, 0, 1, ...]




In [ ]:
#This cell reads the labels of the data and prepares it for the model

def spectrum_label(spectra_data)-> list:
    """
    Assigns labels to spectra based on known modifications in TITLE.

    Parameters:
    - spectra_data (list of dict): List of spectrum dictionaries (from DatasetHandler).

    Returns:
    - List of labels for each spectrum.
    """
    if not isinstance(spectra_data, list):
        print("ERROR: Expected a list of spectra, got", type(spectra_data))
        return None

    labels = []

    for spectrum in spectra_data:
        if not isinstance(spectrum, dict):
            print(f"WARNING: Expected spectrum to be a dict, got {type(spectrum)}")
            labels.append(0)
            continue

        # Get spectrum title (now correctly stored!)
        spectrum_id = spectrum.get("title", "").lower()  # Convert to lowercase for consistency

        if not spectrum_id:
            print(f"WARNING: Missing title for spectrum, assigning label 0")
            labels.append(0)
            continue

        # Assign labels based on keywords in TITLE
        if "oxidation" in spectrum_id:
            labels.append(1)
        elif "phospho" in spectrum_id:
            labels.append(2)
        else:
            labels.append(0)

    print(f"Labels: {Counter(labels)}")
    return labels


---

# 🧠 Hybrid CNN-Transformer Multi-Label Classifier

This module defines the **final architecture** used for **multi-label PTM classification** from MS/MS spectra.
The model integrates **local pattern extraction (CNN)**, **global context modeling (Transformer)**, and **metadata (parent ion mass)**, and predicts each class with an **independent MLP head** (one-vs-rest).

---

## 🔹 `PositionalEncoding`

Implements **sinusoidal positional encodings** (Vaswani et al., 2017), injecting sequence order information into embeddings.

* **Signature:**

  ```python
  PositionalEncoding(d_model: int = 64, seq_len: int = 4500, dropout: float = 0.1)
  ```
* **Behavior:** Precomputes a tensor of `sin`/`cos` terms and adds it to the input, followed by dropout.
* **Input:** `[B, L, d_model]` with `L ≤ seq_len`
* **Output:** Same shape as input

**Example**

```python
pe = PositionalEncoding()
x = torch.randn(32, 10, 64)   # [batch, seq_len, d_model]
x = pe(x)                     # same shape
```

---

## 🔹 `MLPHead`

A compact one-vs-rest head producing a **single logit** per class.

```python
MLPHead(input_dim, hidden_dim=64, dropout=0.5)
# Linear(input_dim → hidden_dim) → ReLU → Dropout → Linear(hidden_dim → 1)
```

---

## 🔹 `EncoderTransformerClassifier`

A **hybrid classifier** with five main blocks:

1. **1D CNN Encoder** – Extracts local spectral patterns

   ```
   Conv1d(1→32, k=5, pad=2) → BN → ReLU
   MaxPool1d(k=2)            # halves length
   Conv1d(32→64, k=3, pad=1) → BN → ReLU
   Flatten
   ```

   * **Output:** `[B, 64 * (input_size // 2)]`

2. **Linear Encoder** – Projects CNN features into Transformer latent space

   ```
   Linear(64*(S/2) → 512) → BN → ReLU
   Linear(512 → latent_size) → BN → ReLU → Dropout
   ```

   * **Output:** `[B, latent_size]`

3. **Positional Encoding + Transformer** – Global context

   * Expand to sequence: `[B, 1, latent_size]`
   * Add sinusoidal encoding
   * `nn.TransformerEncoder` with:

     * `num_layers`, `num_heads`
     * `dim_feedforward = 4 * latent_size`
     * `dropout = dropout_prob`
     * `batch_first=True`, **`norm_first=True`**
   * Mean over sequence dim → `[B, latent_size]`

4. **Parent Ion Processor** – Encodes normalized parent mass

   ```
   Linear(1 → 64) → ReLU
   Linear(64 → latent_size) → ReLU
   ```

   * **Output:** `[B, latent_size]`

5. **Fusion & One-vs-Rest Heads**

   ```
   concat([spectrum, parent]) → [B, 2*latent_size]
   Dropout
   Heads: 3 × MLPHead(2*latent_size → 1 logit)
   ```

   * **Output:** logits `[B, 3]` (concatenated from three heads)

---

### ✅ Forward Pass

**Inputs**

* `spectra`: `[B, S]` (dense binned spectrum, length = `input_size`)
* `parent_ion`: `[B]` (normalized precursor mass)

**Output**

* `logits`: `[B, 3]` — **independent logits per class** (multi-label)

---

### 🔧 Implementation Notes

* `latent_size % num_heads == 0` is enforced.
* `input_size` must be **even** (due to `MaxPool1d`).
* The Transformer currently sees only **one token** per spectrum (a global embedding).
  To enable attention over multiple tokens, feed a sequence (e.g., retain the CNN temporal dimension before flattening).
* The current implementation instantiates **3 heads** explicitly:

  ```python
  self.heads = nn.ModuleList([MLPHead(2*latent_size, hidden_dim=latent_size, dropout=dropout_prob) for _ in range(3)])
  ```

  If you want it to follow `num_classes`, change `range(3)` to `range(num_classes)`.

---

### 🧪 Example

```python
model = EncoderTransformerClassifier(
    input_size=175, latent_size=64, num_classes=3,   # num_classes stored; heads currently fixed to 3
    num_heads=4, num_layers=2, dropout_prob=0.1
)

spectra = torch.randn(32, 175)  # [batch, S]
parent  = torch.rand(32)        # [batch], normalized
logits  = model((spectra, parent))  # [32, 3]
```

**Loss (Multi-label)**

Use **independent** sigmoid + **BCEWithLogitsLoss** with multi-hot targets of shape `[B, 3]`:

```python
targets = torch.tensor([[1,0,1], [0,1,0], ...], dtype=torch.float32)  # multi-hot per sample
loss_fn = nn.BCEWithLogitsLoss()
loss = loss_fn(logits, targets)
```

**Inference (per-class probabilities & thresholds)**

```python
probs = torch.sigmoid(logits)       # [B, 3]
preds = (probs >= 0.5).int()        # thresholdable per class
```


In [ ]:


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int = 64, seq_len: int = 175, dropout: float = 0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)

        position = torch.arange(seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(seq_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # shape: [1, seq_len, d_model]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len]
        return self.dropout(x)


class MLPHead(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.net(x)

class EncoderTransformerClassifier(nn.Module):
    def __init__(self, input_size, latent_size, num_classes, num_heads, num_layers, dropout_prob, max_len=1000):
        super(EncoderTransformerClassifier, self).__init__()
        self.input_size = input_size
        self.latent_size = latent_size
        self.num_classes = num_classes

        # Validate divisibility
        if latent_size % num_heads != 0:
            raise ValueError(f"latent_size ({latent_size}) must be divisible by num_heads ({num_heads}).")

        # 1. CNN Encoder (New Layer)
        self.cnn_encoder = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),  # Downsample
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Flatten()
        )

        # 2. Linear Encoder (Refactored)
        self.encoder = nn.Sequential(
            nn.Linear(64 * (input_size // 2), 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, latent_size),
            nn.BatchNorm1d(latent_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob)
        )

        #3. Positional Encoding
        self.positional_encoding = PositionalEncoding(d_model=latent_size, seq_len=max_len, dropout=dropout_prob)

        # 4. Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=latent_size,
                nhead=num_heads,
                dim_feedforward=latent_size * 4,
                dropout=dropout_prob,
                activation='relu',
                batch_first=True,
                norm_first=True
            ),
            num_layers=num_layers
        )

        # Parent Ion Layer
        self.parent_ion_layer = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, self.latent_size),
            nn.ReLU()
        )

        # Dropout before classification
        self.dropout = nn.Dropout(dropout_prob)

        # One-vs-Rest heads (multi-label)
        self.heads = nn.ModuleList([
            MLPHead(latent_size * 2, hidden_dim=latent_size, dropout=dropout_prob) for _ in range(3)
        ])

    def forward(self, inputs):
        spectra, parent_ion = inputs
        parent_ion = parent_ion.unsqueeze(1)

        # CNN Encoder
        spectra = spectra.unsqueeze(1)  # Ensure input is [B, 1, S]
        cnn_output = self.cnn_encoder(spectra)

        # Linear Encoder
        x = self.encoder(cnn_output)

        # Positional Encoding and Transformer
        x = x.unsqueeze(1)  # Adding sequence dimension
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)

        # Parent Ion Encoding
        parent = self.parent_ion_layer(parent_ion).squeeze(1)

        # Concatenate
        combined = torch.cat([x, parent], dim=1)
        combined = self.dropout(combined)

        # MLP Heads
        outputs = [head(combined) for head in self.heads]
        return torch.cat(outputs, dim=1)


### 🧪 Training, Evaluation & Logging Utilities

This section defines core utility functions used to train, evaluate, and monitor the Transformer-based classifier on MS/MS spectra.

---

#### 🗂️ Logging Setup
Configures a logging pipeline that:
- Removes any pre-existing Colab logging handlers to avoid duplication.
- Logs model performance, evaluation scores, and batch-level information to a persistent file on Google Drive.


---

#### 🧪 `train_classifier_with_weights(...)`

Trains the Transformer classifier using **weighted binary cross-entropy loss** to handle **class imbalance**. Key features:
- Dynamically computes class weights based on training batch distribution.
- Incorporates **L1 regularization** to prevent overfitting and encourage sparsity.
- Logs epoch-wise loss and accuracy.
- Saves model weights to disk upon completion.

Uses **one-hot encoded labels** and tracks prediction accuracy by comparing rounded sigmoid outputs to targets.

---

#### 📊 `evaluate_model(...)`

Evaluates the model on a validation batch with the following features:
- Converts integer labels to multi-hot vectors.
- Applies sigmoid activation to get predicted class probabilities.
- Computes:
  - **Macro** and **Weighted**: Precision, Recall, F1-score
  - **ROC-AUC** and **PR-AUC**
  - Class distribution and per-class metrics via `classification_report`
- Computes a custom **composite score** weighted toward macro F1, PR-AUC, and recall to better reflect performance under class imbalance.

### 📊 Custom Evaluation Score

To reflect the model's performance priorities (especially under class imbalance), we define a **composite evaluation score** as a weighted sum of key metrics:

$$
\text{Score} = 0.40 \cdot \text{Macro-F1} + 0.35 \cdot \text{PR-AUC} + 0.25 \cdot \text{Macro-Recall} + 0.05 \cdot \text{ROC-AUC}
$$
---

### 🧠 Rationale

- **Macro-F1 (40%)**: Emphasizes balanced per-class precision and recall.
- **PR-AUC (35%)**: Prioritizes performance on rare classes (especially PTMs).
- **Macro-Recall (25%)**: Promotes sensitivity (true positive rate) across all classes.
- **ROC-AUC (5%)**: Included for completeness, but de-emphasized due to its limitations under class imbalance.

This scoring approach guides model selection and checkpointing toward better performance on rare and biologically meaningful PTMs like oxidation and phosphorylation.

---

These utilities provide a robust training pipeline with integrated evaluation, threshold tuning (optional), and systematic logging for reproducibility and model comparison.


In [ ]:
#Testing, evaluating and logging cell

#Remove existing handlers to prevent Colab caching issues
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

log_dir = "/content/drive/MyDrive/peak_encoder_transformer_pipeline/logs"
os.makedirs(log_dir, exist_ok=True)

log_path = os.path.join(log_dir, "binning_transformer.log")

# Reapply config AFTER handlers are cleared
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

logger = logging.getLogger("spectra_logger")




#at the moment optimal threshold is not being used, but we are considering introducing it or not on the evalaution fuction
def find_optimal_threshold_v3_multiclass(y_true, y_probs, previous_thresholds=None, beta=2, smooth_alpha=0.1, default_threshold=0.5) -> np.array:
    """
    Finds the best classification threshold for each class based on the Precision-Recall Curve,
    optimizing for F2-score (prioritizing recall) with optional threshold smoothing for multi-class.

    Args:
    - y_true (np.array): Ground truth labels, one-hot encoded or integer labels for multi-class classification.
    - y_probs (np.array): Predicted probabilities, shape (n_samples, n_classes).
    - previous_thresholds (np.array, optional): Previous batch thresholds for smoothing, one threshold per class.
    - beta (float): F-beta score weight for recall (default=2).
    - smooth_alpha (float): EMA smoothing factor (default=0.1).
    - default_threshold (float): Default threshold to return if no valid thresholds are found (default=0.5).

    Returns:
    - np.array: Optimal classification thresholds for each class.
    """
    n_classes = y_probs.shape[1]
    optimal_thresholds = np.zeros(n_classes)

    for class_idx in range(n_classes):
        # Get the true labels for the current class (one-vs-rest)
        y_true_class = (y_true == class_idx).astype(int)  # Convert to binary labels for this class
        y_probs_class = y_probs[:, class_idx]  # Predicted probabilities for this class

        # Calculate Precision-Recall curve for this class
        precision, recall, thresholds = precision_recall_curve(y_true_class, y_probs_class)

        # If no valid thresholds exist, use the default threshold
        if not thresholds.size:
            print(f"No valid thresholds found for class {class_idx}. Returning default threshold: {default_threshold:.4f}")
            optimal_thresholds[class_idx] = default_threshold
            continue

        # Calculate F-beta scores with recall emphasis
        f_beta_scores = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall + 1e-10)

        # Select the threshold corresponding to the best F-beta score
        best_threshold = thresholds[np.argmax(f_beta_scores)]

        # Clip the threshold to ensure it's between 0.05 and 0.95
        best_threshold = np.clip(best_threshold, 0.05, 0.95)

        # Apply Exponential Moving Average (EMA) for smoothing if previous_thresholds is provided
        if previous_thresholds is not None:
            best_threshold = smooth_alpha * best_threshold + (1 - smooth_alpha) * previous_thresholds[class_idx]

        # Store the optimal threshold for this class
        optimal_thresholds[class_idx] = best_threshold

        # Print the selected threshold for this class
        print(f"Class {class_idx} - Selected Threshold (F{beta}-score optimized): {best_threshold:.4f}")

    return optimal_thresholds



def train_classifier_with_weights(model, data_tensors, labels, epochs=100, learning_rate=0.001,l1_lambda=0.001, save_path="model_final_weights.pth", device='cuda'):
    """
    Trains the encoder-classifier model with a weighted loss function for handling class imbalance.
    Assumes all inputs are already PyTorch tensors on the correct device.
    """
    spectra_tensors, parent_ions = data_tensors
    parent_ions = parent_ions.unsqueeze(1)  # Ensure shape (batch_size, 1)
    labels_tensors = labels  # Already a tensor

    # Compute class weights (just once on CPU for Counter)
    class_counts = Counter(labels_tensors.cpu().numpy())
    num_classes = torch.max(labels_tensors).item() + 1
    class_weights = torch.tensor([1.0, 5.0, 2.0], dtype=torch.float32).to(device)

    for cls in range(num_classes):
        if cls in class_counts:
            class_weights[cls] = len(labels_tensors) / (num_classes * class_counts[cls])

    print(f"Class Weights: {class_weights.cpu().numpy()}")

    # Optimizer & Loss
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01) #SGD worth a try?
    loss_fn = nn.BCEWithLogitsLoss()  # or add pos_weight=torch.tensor([...]) if it performs badly

    epoch_losses = []
    epoch_accuracies = []

    # Train loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        outputs = model((spectra_tensors ,parent_ions))
        labels_onehot = F.one_hot(labels_tensors, num_classes=3).float()
        loss = loss_fn(outputs, labels_onehot)

        # L1 Regularization
        l1_loss = 0.0
        for param in model.parameters():
            l1_loss += torch.sum(torch.abs(param))
        loss += l1_lambda * l1_loss

        loss.backward()
        optimizer.step()


        probs = torch.sigmoid(outputs)  # (batch_size, 3)
        predictions = (probs >= 0.5).float()
        accuracy = ((predictions == labels_onehot).float().mean()).item()


        #Store metrics
        epoch_losses.append(loss.item())
        epoch_accuracies.append(accuracy)

        #Log epoch details
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {loss.item():.4f}, Accuracy: {accuracy * 100:.2f}%")

        # Debugging Information
        if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
            print(f"Sample Predictions: {predictions[:5].cpu().numpy()}")
            print(f"Actual Labels: {labels[:5]}")
            print(f"Sample Logits: {outputs[:5].detach().cpu().numpy()}")

    #Save model weights after training
    torch.save(model.state_dict(), save_path)
    print(f"Final model weights saved to {save_path}")



def evaluate_model(model, data_tensors, labels, batch=None):
    """
    Evaluates a One-vs-Rest (multi-label) classifier and computes performance metrics.
    """
    model.eval()
    spectra_tensors, parent_ions = data_tensors
    parent_ions = parent_ions.unsqueeze(1)

    # Convert integer labels (0/1/2) to multi-hot vectors [batch_size, 3]
    if labels.ndim == 1:  # integer labels
        labels = F.one_hot(labels, num_classes=3).float()
    targets = labels  # multi-hot tensor

    with torch.no_grad():
        outputs = model((spectra_tensors, parent_ions))
        loss_fn = nn.BCEWithLogitsLoss()
        loss = loss_fn(outputs, targets)

        # Get sigmoid probabilities
        probabilities = torch.sigmoid(outputs).cpu().numpy()
        predictions = (probabilities >= 0.5).astype(int)
        targets_np = targets.cpu().numpy()

        # Macro & Weighted scores
        macro_precision = precision_score(targets_np, predictions, average='macro', zero_division=0)
        macro_recall = recall_score(targets_np, predictions, average='macro', zero_division=0)
        macro_f1 = f1_score(targets_np, predictions, average='macro', zero_division=0)
        weighted_precision = precision_score(targets_np, predictions, average='weighted', zero_division=0)
        weighted_recall = recall_score(targets_np, predictions, average='weighted', zero_division=0)
        weighted_f1 = f1_score(targets_np, predictions, average='weighted', zero_division=0)

        # ROC & PR AUC
        try:
            roc_auc = roc_auc_score(targets_np, probabilities, average='macro')
        except ValueError:
            roc_auc = float('nan')
        try:
            pr_auc = average_precision_score(targets_np, probabilities, average="macro")
        except ValueError:
            pr_auc = float('nan')

        # Class distribution
        class_distribution = Counter(np.argmax(targets_np, axis=1))

        # Per-class report
        class_names = ["Unmodified", "Oxidation", "Phospho"]
        report = classification_report(targets_np, predictions, target_names=class_names, zero_division=0, digits=4)

        score = 0.4 * macro_f1 + 0.35 * pr_auc + 0.25 * macro_recall + 0.05 * roc_auc
        log_message = (
            f"Batch {batch if batch is not None else '-'}: Validation Loss: {loss.item():.4f},\n"
            f"Macro Precision: {macro_precision:.4f}, Macro Recall: {macro_recall:.4f}, Macro F1-score: {macro_f1:.4f},\n"
            f"Weighted Precision: {weighted_precision:.4f}, Weighted Recall: {weighted_recall:.4f}, Weighted F1-score: {weighted_f1:.4f},\n"
            f"ROC-AUC: {roc_auc:.4f}, PR-AUC: {pr_auc:.4f}\n"
            f"Class Distribution: {class_distribution}\n"
            f"Per-class metrics:\n{report}"
        )

        print(log_message)
        logger.info(log_message)

    return score


here’s a clean markdown you can paste in your thesis that matches the Optuna script you shared.

# Optuna-Based Hyperparameter Tuning (Streaming Mini-Batches)

This section explains how we tune the Transformer classifier’s hyperparameters using **Optuna**, with **low-memory, file-streamed mini-batches** from `.mgf` data.

---

## Configuration

* **Classes:** `num_classes = 3`
* **Data:** `input_dir = "/content/drive/MyDrive/data/balanced_dataset"`
* **Artifacts:** `model_weights_dir = "/content/drive/MyDrive/peak_encoder_transformer_pipeline/model_weights"`
* **Feature binning:** `num_bins = 4500` (fixed in tuning)
* **Precursor normalization:** `pepmass_range = [500, 6000]`, window `200.0`
* **Train epochs per inner run:** `EPOCHS = 60` (in 3 chunks of 20)
* **Device:** auto-select GPU if available
* **Trials:** `n_trials` (e.g., 100)
* **Controller:** `run_optuna = True` (runs tuning; otherwise falls back)

---

## Search Space

We tune the following model and training hyperparameters:

* `latent_size ∈ {64, 128, 256}`
* `num_heads ∈ {2, 4, 8}` (constraint: `latent_size % num_heads == 0`; otherwise prune the trial)
* `num_layers ∈ [2, 6]` (integer)
* `dropout_prob ∈ [0.10, 0.35]` (continuous)
* `l1_lambda ∈ [1e−7, 5e−6]` (log-scaled)
* `learning_rate ∈ [1e−4, 3e−4]` (log-scaled)

> Note: `num_bins` is fixed at 4500 during tuning, but could be made tunable in future work.

---

## Objective & Data Flow (Low-Memory)

To reduce variance while keeping RAM usage small, each trial evaluates the model over **two streamed mini-batches** (i.e., two `.mgf` files), without loading the entire dataset at once.

1. **Trial seeding:** Each trial is deterministically seeded (`seed = trial.number + 1337`), and we seed Python, NumPy, and PyTorch (CPU/GPU).
2. **Batch streaming:**

   * A `DatasetHandler` yields one `.mgf` file per step (`NUM_BATCHES = 2`).
   * We compute features on the fly using `combine_features(...)` to obtain:

     * a **binned spectrum vector** (size = `num_bins`)
     * a **normalized parent ion** value
3. **Per-batch train/val split:**

   * 80/20 split with **stratification** on labels (`train_test_split(..., stratify=labels)`), using `random_state = seed + b`.
4. **Inner training loop:**

   * Fresh model per trial with the candidate hyperparameters.
   * Train for **60 epochs total** in 3 chunks of **20 epochs**, calling:
     `train_classifier_with_weights(model, train_data, y_tr, epochs=20, learning_rate=lr, l1_lambda=l1_lambda, ...)`
5. **Validation & reporting:**

   * After each 20-epoch chunk, compute a **validation score** with `evaluate_model(model, val_data, y_va)` and report it to Optuna (`trial.report(...)`).
   * If the pruner decides to stop early (`trial.should_prune()`), the trial is **pruned** to save compute.

The trial’s objective value is the **mean validation score** across the available mini-batches.

---

## Pruning Strategy

We use Optuna’s **Median Pruner** with `n_warmup_steps = 5`, which compares intermediate results across trials and **early-terminates** underperforming trials after sufficient evidence accumulates. This substantially reduces tuning time.

---

## Study Setup & Visualizations

The study is created to **maximize** the objective (validation score):

* `optuna.create_study(direction="maximize", pruner=MedianPruner(...))`
* `study.optimize(objective, n_trials=n_trials)`

After optimization, we generate standard Optuna diagnostic plots:

* **Parameter Importances**
* **Optimization History**
* **Slice Plot**
* **Parallel Coordinate Plot**

These help identify sensitive hyperparameters and interactions.

---

## Reproducibility & Determinism

* **Per-trial seeds** ensure repeatable splits and training behavior.
* The **stratified** 80/20 split preserves class balance inside each streamed mini-batch.
* The **constraint** `latent_size % num_heads == 0` guarantees a valid multi-head attention configuration; invalid combos are pruned immediately.

---

## Why This Design

* **Memory-efficient:** Only one `.mgf` file’s worth of data is processed at a time.
* **Variance control:** Averaging across two independent streamed batches stabilizes validation estimates without large compute overhead.
* **Compute-aware:** Chunked training with **intermediate evaluations** enables effective **pruning**, focusing effort on promising trials.
* **Portable:** The tuning loop reuses the same feature pipeline and evaluation function as the main training workflow, ensuring consistency between tuning and final training.



In [ ]:


# === CONFIGURATION FLAGS ===
run_optuna = True   # Set to False if you want to run the full training loop instead
n_trials = 100       # Number of Optuna trials

# === FIXED PARAMS ===
num_classes = 3
pepmass_range = {'min': 500.00, 'max': 6000.00}
window_normaliation_size = 200.00
epoch = 100
num_loops = 1
min_score_threshold = 0.92
input_dir = "/content/drive/MyDrive/data/balanced_dataset"
model_weights_dir = "/content/drive/MyDrive/peak_encoder_transformer_pipeline/model_weights"
load_latest_model_at_start = True
excluted_file_name = "split_file_251.mgf"
assume_observed = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

# === OPTUNA OBJECTIVE ===
def objective(trial):
    # --- Hyperparameters to tune ---
    num_bins     = 4500  # keep fixed or make tunable
    latent_size  = trial.suggest_categorical("latent_size", [64, 128, 256])
    num_heads    = trial.suggest_categorical("num_heads", [2, 4, 8])
    if latent_size % num_heads != 0:
        raise optuna.exceptions.TrialPruned()

    num_layers   = trial.suggest_int("num_layers", 2, 6)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.35)
    l1_lambda    = trial.suggest_float("l1_lambda", 1e-7, 5e-6, log=True)
    lr           = trial.suggest_float("learning_rate", 1e-4, 3e-4, log=True)

    # Deterministic seeding per trial
    seed = trial.number + 1337
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

    # --- Evaluate on multiple mini-batches to reduce variance ---
    NUM_BATCHES = 2
    scores = []

    handler = DatasetHandler(input_dir, num_loops=1)

    for b in range(NUM_BATCHES):
        result = handler.get_next_file()
        if result is None:
            break
        spectra_batch, batch_file = result

        feature_batch = combine_features(spectra_batch, pepmass_range,
                                         num_bins, window_normaliation_size,assume_observed)
        if not feature_batch:
            raise optuna.exceptions.TrialPruned()

        spectra, parent_ions = zip(*feature_batch)
        spectra = torch.tensor(np.array(spectra), dtype=torch.float32, device=device)
        parent_ions = torch.tensor(np.array(parent_ions), dtype=torch.float32, device=device)
        labels = torch.tensor(spectrum_label(spectra_batch), dtype=torch.long, device=device)

        # Train/Val split
        indices = np.arange(len(labels))
        tr_idx, va_idx = train_test_split(
            indices, test_size=0.2, stratify=labels.cpu().numpy(), random_state=seed+b
        )

        train_data = (spectra[tr_idx], parent_ions[tr_idx])
        val_data   = (spectra[va_idx], parent_ions[va_idx])
        y_tr, y_va = labels[tr_idx], labels[va_idx]

        # --- Model Init ---
        model = EncoderTransformerClassifier(
            latent_size=latent_size,
            num_heads=num_heads,
            num_layers=num_layers,
            dropout_prob=dropout_prob,
            input_size=num_bins,
            num_classes=num_classes
        ).to(device)

        # --- Training in shorter inner loops ---
        EPOCHS = 60
        for ep in range(0, EPOCHS, 20):
            train_classifier_with_weights(
                model, train_data, y_tr,
                epochs=20, learning_rate=lr, l1_lambda=l1_lambda,
                save_path="/dev/null", device=device
            )
            val_score = evaluate_model(model, val_data, y_va)
            trial.report(val_score, step=ep+20)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        scores.append(val_score)

    if not scores:
        raise optuna.exceptions.TrialPruned()

    return float(np.mean(scores))


# === RUN STUDY ===
if run_optuna:
    study = optuna.create_study(
        direction="maximize",
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)
    )
    study.optimize(objective, n_trials=n_trials)

    print("Best trial:")
    print(study.best_trial.params)

    optuna.visualization.plot_param_importances(study).show()
    optuna.visualization.plot_optimization_history(study).show()
    optuna.visualization.plot_slice(study).show()
    optuna.visualization.plot_parallel_coordinate(study).show()
else:
    # fallback: run your original training loop
    pass


[I 2025-09-15 16:52:23,525] A new study created in memory with name: no-name-6eb210ba-fe5f-46bd-ac20-e3a8a7fbc06f


CUDA available: True
Device: NVIDIA A100-SXM4-40GB
Processing file: /content/drive/MyDrive/data/balanced_dataset/split_file_50.mgf
Total skipped spectra: 0
Labels: Counter({0: 500, 1: 465, 2: 266})


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Class Weights: [0.82      0.8817204 1.5471698]
Epoch [1/20] - Loss: 1.0722, Accuracy: 43.97%
Epoch [2/20] - Loss: 0.9842, Accuracy: 50.81%
Epoch [3/20] - Loss: 0.9476, Accuracy: 54.81%
Epoch [4/20] - Loss: 0.9307, Accuracy: 55.86%
Epoch [5/20] - Loss: 0.8911, Accuracy: 60.23%
Epoch [6/20] - Loss: 0.8835, Accuracy: 61.59%
Epoch [7/20] - Loss: 0.8613, Accuracy: 66.43%
Epoch [8/20] - Loss: 0.8410, Accuracy: 68.36%
Epoch [9/20] - Loss: 0.8330, Accuracy: 68.67%
Epoch [10/20] - Loss: 0.8133, Accuracy: 72.15%
Sample Predictions: [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Actual Labels: tensor([0, 0, 1, 2, 1], device='cuda:0')
Sample Logits: [[-0.3886584   0.35943636 -0.8242121 ]
 [ 0.12890191 -0.50529534 -0.9730778 ]
 [-0.09637618 -0.18061048 -0.15569533]
 [-1.0288622  -0.46865097 -0.36201522]
 [-0.98639196 -1.1411387  -1.0251454 ]]
Epoch [11/20] - Loss: 0.8053, Accuracy: 71.85%
Epoch [12/20] - Loss: 0.7842, Accuracy: 74.39%
Epoch [13/20] - Loss: 0.7701, Accuracy: 75.88%
Epo

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


...

Final model weights saved to /dev/null
Batch -: Validation Loss: 0.9668,
Macro Precision: 0.1366, Macro Recall: 0.3333, Macro F1-score: 0.1938,
Weighted Precision: 0.1680, Weighted Recall: 0.4098, Weighted F1-score: 0.2383,
ROC-AUC: 0.6323, PR-AUC: 0.4721
Class Distribution: Counter({np.int64(0): 100, np.int64(1): 97, np.int64(2): 47})
Per-class metrics:
              precision    recall  f1-score   support

  Unmodified     0.4098    1.0000    0.5814       100
   Oxidation     0.0000    0.0000    0.0000        97
     Phospho     0.0000    0.0000    0.0000        47

   micro avg     0.4098    0.4098    0.4098       244
   macro avg     0.1366    0.3333    0.1938       244
weighted avg     0.1680    0.4098    0.2383       244
 samples avg     0.4098    0.4098    0.4098       244

Processing file: /content/drive/MyDrive/data/balanced_dataset/split_file_206.mgf
Total skipped spectra: 0
Labels: Counter({0: 500, 1: 478, 2: 229})


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Class Weights: [0.8041667 0.8420593 1.7577413]
Epoch [1/20] - Loss: 1.1909, Accuracy: 42.42%
Epoch [2/20] - Loss: 1.0977, Accuracy: 60.48%
Epoch [3/20] - Loss: 1.0524, Accuracy: 67.67%
Epoch [4/20] - Loss: 1.0055, Accuracy: 74.89%
Epoch [5/20] - Loss: 0.9781, Accuracy: 76.06%
Epoch [6/20] - Loss: 0.9463, Accuracy: 76.51%
Epoch [7/20] - Loss: 0.9207, Accuracy: 78.45%
Epoch [8/20] - Loss: 0.8972, Accuracy: 79.41%
Epoch [9/20] - Loss: 0.8693, Accuracy: 80.45%
Epoch [10/20] - Loss: 0.8479, Accuracy: 81.97%
Sample Predictions: [[0. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Actual Labels: tensor([0, 1, 1, 0, 1], device='cuda:0')
Sample Logits: [[-0.6142928  -1.5995106  -2.4989605 ]
 [-1.412734    0.36865047 -1.7435399 ]
 [-1.373682    0.678287   -1.6955684 ]
 [-0.35764456 -1.2419854  -1.8417988 ]
 [-1.0711497  -0.00287712 -1.2464385 ]]
Epoch [11/20] - Loss: 0.8188, Accuracy: 85.35%
Epoch [12/20] - Loss: 0.7925, Accuracy: 86.39%
Epoch [13/20] - Loss: 0.7698, Accuracy: 88.08%
Epo

[I 2025-09-15 17:04:16,807] Trial 99 pruned. 


Final model weights saved to /dev/null
Batch -: Validation Loss: 3.9222,
Macro Precision: 0.1377, Macro Recall: 0.3333, Macro F1-score: 0.1949,
Weighted Precision: 0.1708, Weighted Recall: 0.4132, Weighted F1-score: 0.2417,
ROC-AUC: 0.7010, PR-AUC: 0.5350
Class Distribution: Counter({np.int64(0): 100, np.int64(1): 96, np.int64(2): 46})
Per-class metrics:
              precision    recall  f1-score   support

  Unmodified     0.4132    1.0000    0.5848       100
   Oxidation     0.0000    0.0000    0.0000        96
     Phospho     0.0000    0.0000    0.0000        46

   micro avg     0.4132    0.4132    0.4132       242
   macro avg     0.1377    0.3333    0.1949       242
weighted avg     0.1708    0.4132    0.2417       242
 samples avg     0.4132    0.4132    0.4132       242

Best trial:
{'latent_size': 64, 'num_heads': 4, 'num_layers': 4, 'dropout_prob': 0.22552320822488636, 'l1_lambda': 2.09073908962626e-07, 'learning_rate': 0.00017588221034432413}
